# Sentence Embeddings using Siamese BERT-Networks

Based on the code from: 
- https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py
- https://github.com/UKPLab/sentence-transformers/blob/master/examples/training_nli_bert.py
- https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_clustering.py

This is a simple test application for sentence embeddings: semantic search in norwegian

We have a corpus with various sentences. Then, for a given query sentence, we want to find the most similar sentence in this corpus.

This script outputs for various queries the top 5 most similar sentences in the corpus.


In [1]:
from sentence_transformers import SentenceTransformer, models
import scipy.spatial

In [4]:
# Define a corpus to try the examples

# Corpus with example sentences. Each group of 3 consecutive sentences is taken 
# from the same news article in norwegian.
corpus = ['Legevakten i Trondheim ble onsdag ettermiddag delvis stengt.',
          '58 personer var inntil tirsdag testet for coronaviruset i Norge.',
          'På døren utenfor legevakten i Trondheim blir personer som tror de kan være smittet av coronaviruset, bedt om å vente utenfor og bruke ringeklokken .',
          'Tyrkias militære aksjoner inne i Syria får kritikk fra den norske forsvarsministeren Frank Bakke-Jensen.',
          'Tyrkias forsvarsminister Hulusi Akarg orienterte sine kolleger i NATO om situasjonen i Idlib-regionen på alliansens forsvarsministermøte i Brussel onsdag.',
          'NATO-sjef Jens Stoltenberg fordømte også Syria og Russlands aksjoner, men kommenterte ikke Tyrkias militære handlinger.',
          'De vordende foreldrene er lettet over å få tilbringe natten under et tak.',
          'Kathy (31) er gravid og hjemløs.',
          'Nå har hun bodd på gaten i åtte måneder.'
          ]

# Query sentences:
# Each sentence is taken from the corresponding news article for the corpus.
queries = ['Færre enn fem var på venterommet sammen med denne personen, og de er informert om at det kan ha vært en smittesituasjon.', 
           'Tyrkia får norsk kritikk for Syria-aksjoner.', 
           'I tillegg ser mange ned på oss fordi vi er hjemløse.']


Now we need to choose a model. Choose wisely from this list: 
- https://github.com/UKPLab/sentence-transformers#english-pre-trained-models
- https://huggingface.co/transformers/pretrained_models.html

In [30]:
# Define our models to test

# https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/multilingual-models.md
distiluse_embedder = SentenceTransformer('distiluse-base-multilingual-cased', device='cpu')

# Usual BERT english
# https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md
bert_embedder = SentenceTransformer('bert-base-nli-mean-tokens', device='cpu')

# To use models not defined by the SentenceTransformer we have to do a bit more

# Use multilingual bert uncased (seems not to be recommended by huggingface documentation)
mult_bert_uncased_model = models.BERT('bert-base-multilingual-uncased')
# Apply mean pooling to get one fixed sized sentence vector. You can try different options here.
mult_bert_uncased_pooling = models.Pooling(mult_bert_uncased_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)
mult_bert_uncased_embedder = SentenceTransformer(modules=[mult_bert_uncased_model, mult_bert_uncased_pooling], device='cpu')

# This is recommended instead
mult_distilbert_cased_model = models.DistilBERT('distilbert-base-multilingual-cased')
# Apply mean pooling to get one fixed sized sentence vector
mult_distilbert_cased_pooling = models.Pooling(mult_distilbert_cased_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)
mult_distilbert_uncased_embedder = SentenceTransformer(modules=[mult_distilbert_cased_model, mult_distilbert_cased_pooling], device='cpu')


Define auxiliary functions to process the data. We want to see the clusters also, to see how these models capture meaning.

In [ ]:
from sklearn.cluster import KMeans

# For clustering, it is recommended these models:
# https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/wikipedia-sections-models.md
def get_top_5_closest(queries, corpus, query_embeddings, corpus_embeddings, top_n=5):
    closest_n = top_n
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])

        print("\n\n======================\n\n")
        print("Query:", query)
        print("\nTop 5 most similar sentences in corpus:")

        for idx, distance in results[0:closest_n]:
            print(corpus[idx].strip(), "(Score: %.4f)" % (1-distance))

def get_cluster_embeddings(corpus_embeddings, num_clusters=5):
    clustering_model = KMeans(n_clusters=num_clusters)
    clustering_model.fit(corpus_embeddings)
    return clustering_model

def print_clusters(clustering_model, corpus, num_clusters=5):
    cluster_assignment = clustering_model.labels_
    clustered_sentences = [[] for i in range(num_clusters)]
    for sentence_id, cluster_id in enumerate(cluster_assignment):
        clustered_sentences[cluster_id].append(corpus[sentence_id])

    for i, cluster in enumerate(clustered_sentences):
        print("Cluster ", i+1)
        print(cluster)
        print("")

Now we create the embeddings for the queries and the corpus for each embedder:

In [7]:
distiluse_corpus_embeddings = distiluse_embedder.encode(corpus)
distiluse_query_embeddings = distiluse_embedder.encode(queries)

bert_corpus_embeddings = bert_embedder.encode(corpus)
bert_query_embeddings = bert_embedder.encode(queries)

mult_bert_uncased_corpus_embeddings = mult_bert_uncased_embedder.encode(corpus)
mult_bert_uncased_query_embeddings = mult_bert_uncased_embedder.encode(queries)

mult_distilbert_uncased_corpus_embeddings = mult_distilbert_uncased_embedder.encode(corpus)
mult_distilbert_uncased_query_embeddings = mult_distilbert_uncased_embedder.encode(queries)

# Distiluse results

In [9]:
get_top_5_closest(queries, corpus, distiluse_query_embeddings, distiluse_corpus_embeddings, top_n=5)





Query: Færre enn fem var på venterommet sammen med denne personen, og de er informert om at det kan ha vært en smittesituasjon.

Top 5 most similar sentences in corpus:
På døren utenfor legevakten i Trondheim blir personer som tror de kan være smittet av coronaviruset, bedt om å vente utenfor og bruke ringeklokken . (Score: 0.6394)
Nå har hun bodd på gaten i åtte måneder. (Score: 0.5088)
De vordende foreldrene er lettet over å få tilbringe natten under et tak. (Score: 0.4775)
Legevakten i Trondheim ble onsdag ettermiddag delvis stengt. (Score: 0.3362)
58 personer var inntil tirsdag testet for coronaviruset i Norge. (Score: 0.2567)




Query: Tyrkia får norsk kritikk for Syria-aksjoner.

Top 5 most similar sentences in corpus:
Tyrkias militære aksjoner inne i Syria får kritikk fra den norske forsvarsministeren Frank Bakke-Jensen. (Score: 0.6853)
NATO-sjef Jens Stoltenberg fordømte også Syria og Russlands aksjoner, men kommenterte ikke Tyrkias militære handlinger. (Score: 0.5618)
Nå 

In [10]:
print_clusters(get_cluster_embeddings(distiluse_corpus_embeddings), corpus)

Cluster  1
['Legevakten i Trondheim ble onsdag ettermiddag delvis stengt.', 'På døren utenfor legevakten i Trondheim blir personer som tror de kan være smittet av coronaviruset, bedt om å vente utenfor og bruke ringeklokken .', 'De vordende foreldrene er lettet over å få tilbringe natten under et tak.', 'Nå har hun bodd på gaten i åtte måneder.']

Cluster  2
['Tyrkias militære aksjoner inne i Syria får kritikk fra den norske forsvarsministeren Frank Bakke-Jensen.', 'NATO-sjef Jens Stoltenberg fordømte også Syria og Russlands aksjoner, men kommenterte ikke Tyrkias militære handlinger.']

Cluster  3
['58 personer var inntil tirsdag testet for coronaviruset i Norge.']

Cluster  4
['Kathy (31) er gravid og hjemløs.']

Cluster  5
['Tyrkias forsvarsminister Hulusi Akarg orienterte sine kolleger i NATO om situasjonen i Idlib-regionen på alliansens forsvarsministermøte i Brussel onsdag.']



# BERT results

In [11]:
get_top_5_closest(queries, corpus, bert_query_embeddings, bert_corpus_embeddings, top_n=5)





Query: Færre enn fem var på venterommet sammen med denne personen, og de er informert om at det kan ha vært en smittesituasjon.

Top 5 most similar sentences in corpus:
De vordende foreldrene er lettet over å få tilbringe natten under et tak. (Score: 0.8958)
Nå har hun bodd på gaten i åtte måneder. (Score: 0.8429)
Tyrkias forsvarsminister Hulusi Akarg orienterte sine kolleger i NATO om situasjonen i Idlib-regionen på alliansens forsvarsministermøte i Brussel onsdag. (Score: 0.8236)
Legevakten i Trondheim ble onsdag ettermiddag delvis stengt. (Score: 0.8192)
Tyrkias militære aksjoner inne i Syria får kritikk fra den norske forsvarsministeren Frank Bakke-Jensen. (Score: 0.7443)




Query: Tyrkia får norsk kritikk for Syria-aksjoner.

Top 5 most similar sentences in corpus:
Tyrkias militære aksjoner inne i Syria får kritikk fra den norske forsvarsministeren Frank Bakke-Jensen. (Score: 0.8748)
NATO-sjef Jens Stoltenberg fordømte også Syria og Russlands aksjoner, men kommenterte ikke Ty

In [12]:
print_clusters(get_cluster_embeddings(bert_corpus_embeddings), corpus)

Cluster  1
['De vordende foreldrene er lettet over å få tilbringe natten under et tak.', 'Nå har hun bodd på gaten i åtte måneder.']

Cluster  2
['58 personer var inntil tirsdag testet for coronaviruset i Norge.', 'På døren utenfor legevakten i Trondheim blir personer som tror de kan være smittet av coronaviruset, bedt om å vente utenfor og bruke ringeklokken .']

Cluster  3
['Tyrkias militære aksjoner inne i Syria får kritikk fra den norske forsvarsministeren Frank Bakke-Jensen.', 'Tyrkias forsvarsminister Hulusi Akarg orienterte sine kolleger i NATO om situasjonen i Idlib-regionen på alliansens forsvarsministermøte i Brussel onsdag.', 'NATO-sjef Jens Stoltenberg fordømte også Syria og Russlands aksjoner, men kommenterte ikke Tyrkias militære handlinger.']

Cluster  4
['Legevakten i Trondheim ble onsdag ettermiddag delvis stengt.']

Cluster  5
['Kathy (31) er gravid og hjemløs.']



# Multilingual BERT uncased

In [13]:
get_top_5_closest(queries, corpus, mult_bert_uncased_query_embeddings, mult_bert_uncased_corpus_embeddings, top_n=5)





Query: Færre enn fem var på venterommet sammen med denne personen, og de er informert om at det kan ha vært en smittesituasjon.

Top 5 most similar sentences in corpus:
På døren utenfor legevakten i Trondheim blir personer som tror de kan være smittet av coronaviruset, bedt om å vente utenfor og bruke ringeklokken . (Score: 0.8778)
58 personer var inntil tirsdag testet for coronaviruset i Norge. (Score: 0.7968)
Legevakten i Trondheim ble onsdag ettermiddag delvis stengt. (Score: 0.7744)
Tyrkias militære aksjoner inne i Syria får kritikk fra den norske forsvarsministeren Frank Bakke-Jensen. (Score: 0.7743)
Nå har hun bodd på gaten i åtte måneder. (Score: 0.7732)




Query: Tyrkia får norsk kritikk for Syria-aksjoner.

Top 5 most similar sentences in corpus:
Tyrkias militære aksjoner inne i Syria får kritikk fra den norske forsvarsministeren Frank Bakke-Jensen. (Score: 0.8851)
NATO-sjef Jens Stoltenberg fordømte også Syria og Russlands aksjoner, men kommenterte ikke Tyrkias militære 

In [27]:
print_clusters(get_cluster_embeddings(mult_bert_uncased_corpus_embeddings), corpus)

Cluster  1
['Kathy (31) er gravid og hjemløs.', 'Nå har hun bodd på gaten i åtte måneder.']

Cluster  2
['Tyrkias militære aksjoner inne i Syria får kritikk fra den norske forsvarsministeren Frank Bakke-Jensen.', 'Tyrkias forsvarsminister Hulusi Akarg orienterte sine kolleger i NATO om situasjonen i Idlib-regionen på alliansens forsvarsministermøte i Brussel onsdag.', 'NATO-sjef Jens Stoltenberg fordømte også Syria og Russlands aksjoner, men kommenterte ikke Tyrkias militære handlinger.']

Cluster  3
['58 personer var inntil tirsdag testet for coronaviruset i Norge.', 'På døren utenfor legevakten i Trondheim blir personer som tror de kan være smittet av coronaviruset, bedt om å vente utenfor og bruke ringeklokken .']

Cluster  4
['Legevakten i Trondheim ble onsdag ettermiddag delvis stengt.']

Cluster  5
['De vordende foreldrene er lettet over å få tilbringe natten under et tak.']



# Multilingual DistilBERT cased - Recommended

In [28]:
get_top_5_closest(queries, corpus, mult_distilbert_uncased_query_embeddings, mult_distilbert_uncased_corpus_embeddings, top_n=5)





Query: Færre enn fem var på venterommet sammen med denne personen, og de er informert om at det kan ha vært en smittesituasjon.

Top 5 most similar sentences in corpus:
På døren utenfor legevakten i Trondheim blir personer som tror de kan være smittet av coronaviruset, bedt om å vente utenfor og bruke ringeklokken . (Score: 0.8656)
58 personer var inntil tirsdag testet for coronaviruset i Norge. (Score: 0.8468)
NATO-sjef Jens Stoltenberg fordømte også Syria og Russlands aksjoner, men kommenterte ikke Tyrkias militære handlinger. (Score: 0.7933)
De vordende foreldrene er lettet over å få tilbringe natten under et tak. (Score: 0.7905)
Tyrkias militære aksjoner inne i Syria får kritikk fra den norske forsvarsministeren Frank Bakke-Jensen. (Score: 0.7851)




Query: Tyrkia får norsk kritikk for Syria-aksjoner.

Top 5 most similar sentences in corpus:
Tyrkias militære aksjoner inne i Syria får kritikk fra den norske forsvarsministeren Frank Bakke-Jensen. (Score: 0.9023)
NATO-sjef Jens S

In [29]:
print_clusters(get_cluster_embeddings(mult_distilbert_uncased_corpus_embeddings), corpus)

Cluster  1
['Kathy (31) er gravid og hjemløs.']

Cluster  2
['Tyrkias militære aksjoner inne i Syria får kritikk fra den norske forsvarsministeren Frank Bakke-Jensen.', 'Tyrkias forsvarsminister Hulusi Akarg orienterte sine kolleger i NATO om situasjonen i Idlib-regionen på alliansens forsvarsministermøte i Brussel onsdag.', 'NATO-sjef Jens Stoltenberg fordømte også Syria og Russlands aksjoner, men kommenterte ikke Tyrkias militære handlinger.']

Cluster  3
['De vordende foreldrene er lettet over å få tilbringe natten under et tak.', 'Nå har hun bodd på gaten i åtte måneder.']

Cluster  4
['58 personer var inntil tirsdag testet for coronaviruset i Norge.']

Cluster  5
['Legevakten i Trondheim ble onsdag ettermiddag delvis stengt.', 'På døren utenfor legevakten i Trondheim blir personer som tror de kan være smittet av coronaviruset, bedt om å vente utenfor og bruke ringeklokken .']

